In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px

# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/Users/arturopolancolozano/opt/anaconda3/envs/finrl_env/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

In [2]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



In [3]:
TRAIN_START_DATE = '2001-01-01'
TRAIN_END_DATE = '2019-05-31'
TRADE_START_DATE = '2019-06-01'
TRADE_END_DATE = '2019-11-30'

In [4]:
#selected_companies_list = ["EC", "CIB", "ARGO"]
selected_companies_list = ["EC"]


df = YahooDownloader(
    start_date = TRAIN_START_DATE,
    end_date = TRADE_END_DATE,
    ticker_list = selected_companies_list, 
).fetch_data()

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2820, 8)


In [5]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2008-09-18,23.00,26.299999,22.400000,10.116409,371500,EC,3
1,2008-09-19,27.35,27.350000,24.750000,11.138269,124100,EC,4
2,2008-09-22,27.50,27.850000,25.350000,11.015645,140200,EC,0
3,2008-09-23,26.25,26.490000,24.500000,10.198156,122500,EC,1
4,2008-09-24,25.25,25.250000,23.700001,9.993787,48300,EC,2


In [6]:

prices = pd.DataFrame(
    {
        "Bancolombia": df.query("tic == 'CIB'")["close"],
        "Ecopetrol": df.query("tic == 'EC'")["close"],
        "Cementos Argos": df.query("tic == 'ARGO'")["close"]

    }
)

prices.index = pd.to_datetime(df.date)
fig = px.area(
    data_frame=prices,
    title = "Precio Cierre Acciones "
)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=10, label="10 Years", step="year", stepmode="backward"),
            dict(count=5, label="5 Years", step="year", stepmode="backward"),
            dict(count=1, label="1 Year", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

# Part 4: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

In [7]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2819, 8)
Successfully added vix
Successfully added turbulence index


In [8]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

processed_full.sort_values(['date','tic'],ignore_index=True).head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2008-09-18,EC,23.00,26.299999,22.400000,10.116409,371500.0,3.0,0.000000,12.072468,9.182211,100.000000,66.666667,100.000000,10.116409,10.116409,33.099998,0.0
1,2008-09-19,EC,27.35,27.350000,24.750000,11.138269,124100.0,4.0,0.022926,12.072468,9.182211,100.000000,66.666667,100.000000,10.627339,10.627339,32.070000,0.0
2,2008-09-22,EC,27.50,27.850000,25.350000,11.015645,140200.0,0.0,0.025410,11.872678,9.640871,88.956960,64.927697,100.000000,10.756775,10.756775,33.849998,0.0
3,2008-09-23,EC,26.25,26.490000,24.500000,10.198156,122500.0,1.0,-0.003598,11.685864,9.548376,50.498333,-24.215579,23.992755,10.617120,10.617120,35.720001,0.0
4,2008-09-24,EC,25.25,25.250000,23.700001,9.993787,48300.0,2.0,-0.027417,11.572961,9.411946,45.420026,-79.591995,10.302811,10.492453,10.492453,35.189999,0.0


In [9]:
mvo_df = processed_full.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]

<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [10]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

2692
126


In [11]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 1, State Space: 11


In [12]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100_000,
    "initial_amount": 1_000_000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

timesteps_value = 1_000_000

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [13]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [14]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

# set up logger
tmp_path = RESULTS_DIR + '/a2c'
new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
model_a2c.set_logger(new_logger_a2c)

trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=timesteps_value)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c
--------------------------------------
| time/                 |            |
|    fps                | 807        |
|    iterations         | 100        |
|    time_elapsed       | 0          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -1.45      |
|    explained_variance | 0.0463     |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | 0.0584     |
|    reward             | -1.7176487 |
|    std                | 1.03       |
|    value_loss         | 6.3        |
--------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 868       |
|    iterations         | 200       |
|    time_elapsed       | 1         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy

### Agent 2: DDPG

### Agent 3: PPO

Testing

In [15]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)

df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c,
    environment = e_trade_gym)



hit end!


In [16]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
df_actions_a2c = df_actions_a2c.set_index(df_actions_a2c.columns[0])
df_actions_a2c.actions = df_actions_a2c.actions.apply(lambda x: x[0])

fig = px.area(
  data_frame=df_result_a2c,
  title = "Resultado Portafolio Trading Bot Ecopetrol"
)
fig.show()



In [17]:
trade_data = trade[["date", "close"]]
trade_data.set_index("date", inplace=True)

fig = px.area(
  data_frame=trade_data,
  title = "Comportamiento Acción Ecopetrol"
)
fig.show()

In [18]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "Rendimiento Trading Bot Ecopetrol", 
        "Comportamiento Acción Ecopetrol")
)

fig.add_trace(
    go.Scatter(
        x=df_result_a2c.index, 
        y=df_result_a2c.account_value,
        name="Bot"
        ),
    row=1, 
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=trade_data.index, 
        y=trade_data.close,
        name="Acción"
        ),
    row=1, 
    col=2
)



fig.update_layout(
    height=500, 
    width=850,
    title_text="Comportamiento Inversion Trading Bot Ecopetrol"
)

fig.show()

In [19]:
df_actions_a2c.value_counts()

actions
0          124
84495        1
Name: count, dtype: int64

In [20]:
df_actions_a2c_plot = df_actions_a2c.reset_index()
df_actions_a2c_plot.date = pd.to_datetime(df_actions_a2c_plot.date)

In [21]:
df_actions_a2c_plot

,date,actions
0,2019-06-03,84495
1,2019-06-04,0
2,2019-06-05,0
3,2019-06-06,0
4,2019-06-07,0
...,...,...
120,2019-11-20,0
121,2019-11-21,0
122,2019-11-22,0
123,2019-11-25,0


In [22]:
fig = px.histogram(
  data_frame=df_actions_a2c_plot,
  x = "date",
  
  title = "Resultado Portafolio Trading Bot Ecopetrol"
)
fig.show()